# Display of List of Available Widgets to Display 

## Overview 

The visualizer is planned to support three visualization modes:
1. Visualize updates to a standalone kvlang script
2. Visualize updates to an existing kivy widget
3. Visualize updates to an existing kivy application 

To allow the user to choose what to visualize, we should populate a listbox that displays each of the visualization options. The user will be able to select which item they would like to visualize. Switching between items will update the visualization. 

## Research

### How do you search a py file for class definitions?
We need to find all App and Widget descendants within each python file. We cannot expect all widget types to be a direct descendant of known kivy types. 

The `inspect` python module will return all of the base classes of a type, but this module requires live objects. We want to avoid executing the user's code, because this will require using their environment and we are trying to keep a clear separation between their environment and the kivy designer environment - plus actually executing their code can have some side effects. We should only do this execution during visualization, when they request it.

Unfortunately we cannot we use the `ast` module to find all objects within a python file that descend from the `App` and `widget` class. The reason is that the AST tree only stores the direct class parents, and does not provide any reference to where those parents are declared. To build a more complete hierarchy tree we would need to search through all the available source files and build the hierarchy ourselves. 


In [ ]:
import ast

# This code fails because ast does not store detailed information
# about ClassDef parents

def get_subclasses_of_widget(filename):
    subclasses = []
    with open(filename, "r") as f:
        tree = ast.parse(f.read())
        for node in ast.iter_child_nodes(tree):
            if isinstance(node, ast.ClassDef):
                if is_derived_from_widget(node):
                    subclasses.append(node.name)
    return subclasses

def is_derived_from_widget(node):
    for base in node.bases:
        if isinstance(base, ast.Name) and base.id == "Widget":
            return True
        elif is_derived_from_widget(base): #ERROR. Can't recursively traverse the tree this way. Base has no attr bases.
            return True
    return False

subclasses = get_subclasses_of_widget("C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\kdfilechooser.py")
print(subclasses)


### What entries should the listbox contain?

(In this order)
1. Names of custom user defined applications
2. Names of custom user defined kvlang scripts
3. Names of custom user defined widgets
4. Names of kivy standard library widgets

In the future we should also populate the listbox with kivy garden widgets. 

### How should the entries be displayed?

A treeview would be appropriate here, since this will allow us to categorize the widgets into groups. 

Unfortunately, we probably won't be able to reuse some of the styling work we did on the kdfileviewer since the file viewer has a complex interplay of objects - but we can apply the same learnings here at the expense of some duplication. 

### How are custom attribute types declared?

To-Do here

## General Solution Plan



1. Add a WidgetScanner class that can recursively search a file directory for `Widget` and `App` classes. The search should avoid compiling any code, so our plan is to use the abstract syntax tree
    * The WidgetScanner class should accept the root file directory
    * It should return the module filepath for each class type
    * It should provide the four groups of widget types specified in the research section (kvfiles, user Apps, user widgets, kivy widgets)
    * It should provide a mechanism to rescan file(s) on update, without rescanning the entire directory
2. Use the WidgetScanner class to find all of the `Widget` and `App` within the `kivy` standard library (for the current version for now), and store these class names in a list
3. Use the list of kivy standard library classes to improve the search within WidgetScanner. The WidgetScanner doesn't need to search up to the widget. It can stop at any of the standard library classes
4. Our search will not be perfect since the ast does not provide detailed information about the parents. Since the ast does not state where the parent class is declared, our imperfect search will have a very small probability of returning false positive (if there is another class in another namespace with the same name as a kivy standard library widget), and we risk missing widgets if the widget descends from a third-party widget that is not present in the available source files. To account for these rare failures, we should provide attributes to explicitely enable and disable the visualization (@disable_kv_vis, @enable_kv_vis)
5. Use a tree view to display the four groups
  * Let's stylize the tree in a separate issue. I'd like to wait until the visualizer is able to help us here 
6. Add a on_submit event handler that will send the proper update instruction to the visualizer whenever an item is selected. Create some dummy instructions for now since hot reloading for class types is not yet implemented

## Implementation Questions

### How should `WidgetScanner` search a directory for widgets and apps?

There are two ways we could approach this problem, using the `ast` to avoid inspecting compiled code:
1. Build an inheritance graph using the source code from each of the source code files. An inheritance graph will have node connections across several source files, so each time a source code file is updated we will need to either rebuild the tree or cache additional information about each source file that will allow us to efficiently update the inheritance graph. Constructing an inheritance graph may also be expensive to build because python's inheritance rules are very loose - for example a class can inherit from a parent and that parent's parent. 

2. Determine whether a class is a kivy widget/app as soon as it is encountered. This will require visiting the parent modules as soon as they are encountered. We could use memoization to reduce excessive source code compile calls. We can also limit the visited modules to files in the project. If we use this structure then we can update & refresh one file at a time without building an overall tree at each refresh. 
  * This way avoids the complexity of building a tree, but adds the complexity of needing to determine where ClassDef source code definitions are located using import statements. This is all technically possible (using functionality like `importlib.util.find_spec()`), but parsing import statements will be challenging and conducting multiple file parses per file could get expensive. 

When we encounter a ClassDef that has parents we will know nothing about that parent unless we've encountered it before. To determine if the parent is a widget/app we need to store its inheritance information in an inheritance tree. If we later find the parent's class definition contains a known widget/app or known non_kivy_class then we will know the child's status as well.

The tree only needs to contain nodes with an unknown top-level ancestor. If the class is known to have no parents, or is known to be a kivy widget/app then simply add them to the known sets and move on.
0. Create three sets: known_app_classes, known_widget_classes, known_non_kivy_classes
1. Define node visitor to visit each ClassDef
2. The ClassDef entry will represent a node on the inheritance graph - complete will all the parents for that class. 
3. Determine if the current entry is a known widget or app. If it is, then append it to the set of known widgets/apps. If the class has known_non_kivy_class parent then move it to that set and do not add it to the tree. 
4. Update the inheritance graph to:
  i) Search the tree to determine if any of the parents are currently in the graph. 
    If they are then add the current node as a child. 
    If they are not, then add each parent node with current node as a child
  ii) Search the tree to determine if the current entry is already present in the graph (without parents, of course). If the node is present, then two actions are possible:
    I) If the current entry is a known widget/app then we now know that all of the children are also. Add them to the sets & trim them from the tree.
    II) If the entry is not a known widget, then update the pre-existing node's parents to reflect the current entry.  

Write a function that returns (app, widget, kvfile) lists.

Make sure the search uses the new attributes


### How should `WidgetScanner` store the widget types along with the filepath, to allow file-by-file update?




In [14]:
import ast 
from pathlib import Path

FILE_1 = '''
class Parent:
    pass 

class Child(Parent):
    pass

class Grandchild(Parent):
    pass
'''
FILE_2 = '''
class Grandchild(Parent):
    pass

class Child(Parent):
    pass

class Parent:
    pass 
'''

'''
        self.maybe_newline()
        for deco in node.decorator_list:
            self.fill("@")
            self.traverse(deco)
        self.fill("class " + node.name)
        with self.delimit_if("(", ")", condition = node.bases or node.keywords):
            comma = False
            for e in node.bases:
                if comma:
                    self.write(", ")
                else:
                    comma = True
                self.traverse(e)
            for e in node.keywords:
                if comma:
                    self.write(", ")
                else:
                    comma = True
                self.traverse(e)
'''

# To do: Rewrite as named tuple
class ClassDefNode:
    def __init__(self, name, parents, children = list()):
        self.name = name
        self.parents = parents
        self.children = children

    def __hash__(self) -> int:
        return hash(self.name)

class InheritanceGraphs:
    def __init__(self) -> None:
        self.nodes = dict()

    def add_classdef(self, classname, parent_classnames):
        '''
        Add a class definition and parents to the graph. 
        If the class already exists, update its parents.
        If the parents already exist, update their children.
        Ignore duplicate entries. Enforce that a class can only have one set of parents.
        '''
        class_node = self.nodes.get(classname)
        if class_node:
            if len(class_node.parents) == 0:
                class_node.parents = parent_classnames
            elif class_node.parents == parent_classnames:
                # Duplicate entry. Ignore and exit. 
                return
            else:
                raise Exception("Class already exists with different parents.")
        else:
            class_node = ClassDefNode(classname, parent_classnames)
            self.nodes[classname] = class_node

        for parent_classname in parent_classnames:
            parent_node = self.nodes.get(parent_classname)
            if parent_node:
                parent_node.children.append(class_node.name)
            else:
                parent_node = ClassDefNode(parent_classname, list(), [class_node.name])
                self.nodes[parent_classname] = parent_node

    def get_subclasses(self, classname):
        '''
        Return a list of all subclasses of the given class.
        '''
        subclasses = list()
        class_node = self.nodes.get(classname)
        if class_node:
            subclasses = class_node.children
            for child in class_node.children:
                subclasses.extend(self.get_subclasses(child))
        return subclasses

    def get_superclasses(self, classname):
        '''
        Return a list of all superclasses of the given class.
        '''
        superclasses = list()
        class_node = self.nodes.get(classname)
        if class_node:
            superclasses = class_node.parents
            for parent in class_node.parents:
                superclasses.extend(self.get_superclasses(parent))
        return superclasses

    def get_all_classes(self):
        return self.nodes.keys()

    def remove_class(self, classname):
        '''
        Remove the given class from the graph.
        '''
        class_node = self.nodes.get(classname)
        if class_node:
            for parent in class_node.parents:
                parent_node = self.nodes.get(parent)
                parent_node.children.remove(class_node)
            for child in class_node.children:
                child.parents.remove(class_node)
            del self.nodes[classname]
    
    def remove_class_and_subclasses(self, classname):
        '''
        Remove the given class and all its subclasses from the graph.
        '''
        self.remove_class(classname)
        for subclass in self.get_subclasses(classname):
            self.remove_class(subclass)


class InheritanceGraphsBuilder(ast.NodeVisitor):

    def __init__(self) -> None:
        self.graphs = InheritanceGraphs()

    def visit_ClassDef(self, node):
        self.graphs.add_classdef(node.name, [base.id for base in node.bases])

    def build(self, file_source):
        tree = ast.parse(file_source)
        self.visit(tree)
        return self.graphs

    def build_from_file(self, filepath):
        file_source = Path(filepath).read_text()
        return self.build(file_source)

    def build_from_directory(self, directory):
        for filepath in Path(directory).rglob('**/*.py'):
            print(filepath)
            self.build_from_file(filepath)

EX_FILE = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\kdfilechooser.py"
EX_FILE1 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\iconbutton.py"
EX_FILE2 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\modalmsg.py"
EX_FILE3 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\resources.py"
EX_FILE4 = "C:\\Users\\joshu\\source\\repos\\kivydesigner\\kivydesigner\\uix\\toolbar.py"
'''
builder = InheritanceGraphsBuilder()
builder.build_from_file(EX_FILE)
builder.build_from_file(EX_FILE1)
builder.build_from_file(EX_FILE2)
builder.build_from_file(EX_FILE3)
builder.build_from_file(EX_FILE4)
builder.build_from_file(EX_FILE)
builder.build_from_file(EX_FILE1)
builder.build_from_file(EX_FILE2)
builder.build_from_file(EX_FILE3)
builder.build_from_file(EX_FILE4)'''

builder = InheritanceGraphsBuilder()
builder.build_from_directory("C:\\Users\\joshu\\source\\repos\\kivydesigner")
print(builder.graphs.get_all_classes())
print("subclasses!")
for classname in builder.graphs.get_all_classes():
    print(classname, builder.graphs.get_subclasses(classname))

print("superclasses!")
for classname in builder.graphs.get_all_classes():
    print(classname, builder.graphs.get_superclasses(classname))

C:\Users\joshu\source\repos\kivydesigner\main.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\hotreload.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\kivydesigner.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\register_uix.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\__init__.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\tests\common.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\tests\test_common.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\tests\test_iconbutton.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\tests\__init__.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\uix\iconbutton.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\uix\kdfilechooser.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\uix\kivyvisualizer.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\uix\modalmsg.py
C:\Users\joshu\source\repos\kivydesigner\kivydesigner\uix\resources.py
C:\Users\joshu\sou

AttributeError: 'Attribute' object has no attribute 'id'